## Fooling with Word Vectors ##
https://github.com/aparrish/rwet/blob/master/understanding-word-vectors.ipynb

https://github.com/aparrish/rwet/blob/master/understanding-word-vectors.ipynb

In order to get some of my Dark Souls objects to rhyme I'll have to learn Word Vectors.

Word Vectors are distance relationships (their *Euclidean distance*) between points. These points can be words and their attributes. 

Here's some basic arithmetic function. 
>(The ** operator raises the value on its left to the power on its right.)



In [1]:
import math
def distance2d(x1, y1, x2, y2):
    return math.sqrt((x1 -x2)**2 + (y1 - y2)**2)

In [2]:
distance2d(70, 30, 75, 40)

11.180339887498949

In [3]:
import json

In [4]:
color_data = json.loads(open("xkcd.json").read())

This code will convert the color from the hex format to a tuple of integers - hence the "#" (#1a2b3c)

In [6]:
def hex_to_int(s):
    s = s.lstrip("#")
    return int(s[:2], 16), int(s[2:4], 16), int(s[4:6],16)

The following will create a dictionary and populate it with mapping from color names to RGB

"Color", "Colors" and "Hex" relates to how the xkcd.json file is organized.

In [7]:
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

In [8]:
colors['olive']

(110, 117, 14)

Create the functions that allow for Vector arithmetic to find distances between points in several dimensions (trippy!)

Tried on paper...using a ruler. Math still works. Here's what's happening: √(x2-x1)²+(y2-y1)²

No idea what's the "zip" for though...

**Distance between points**

In [9]:
def distance(coord1, coord2):
    # VERY SLOW - use for learning purposed only!
    return math.sqrt(sum([(i-j)**2 for i, j in zip(coord1, coord2)]))

In [ ]:
distance([10,1], [5,2])

**Subtract Vectors**

In [12]:
def subtractv (coord1, coord2):
    return [c1 - c2 for c1, c2 in zip (coord1, coord2)]

In [13]:
subtractv([10,1],[5,2])

[5, -1]

**Add Vectors**

In [14]:
def addv(coord1, coord2):
    return[c1 + c2 for c1, c2 in zip(coord1, coord2)]
addv([10,1], [5,2])

[15, 3]

**Average between a list of vectors**

In [42]:
def meanv(coords):
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean
meanv([[0, 1], [2, 2], [4, 3]])

[2.0, 2.0]

In [44]:
distance(colors['red'], colors['dark pastel green'])

241.4415043028021

In [46]:
distance(colors['blue'], colors['green']) > distance(colors['red'], colors['pink'])

False

## Word Vectors in spaCY ##

The xkcd.json example dealt with colors that were already organized as a numbered matrix (don't even know if this makes sense, but bear with me). But words are not so easily related. The number of dimensions vary imensily and change depending on methodology. 

Allison recommends in her tutorial to go straight to using spaCy. This program uses **GloVe** - Stanford's ready made (trained) word vectors.

In [47]:
import spacy

In [56]:
nlp = spacy.load('en_core_web_md')


In [72]:
file = open(filename, encoding="utf8")


SyntaxError: invalid syntax (<ipython-input-72-99e161da019a>, line 1)

In [74]:
doc = nlp(open("frankenstein.txt", encoding="utf8").read())

Grabbed frankenstein (84-0.txt) from the gutenberg page, renamed it and ran using the ", encoding="utf8" command - Withouth the charset I had errors.

In [75]:
# add all the words in the text file
tokens = list(set([w.text for w in doc if w.is_alpha]))

In [80]:
nlp.vocab['master'].vector

array([  2.65439987e-01,  -4.81970012e-01,   4.41280007e-01,
        -6.79279983e-01,   4.23440002e-02,  -2.80189998e-02,
        -7.59750009e-02,  -7.15219975e-01,  -4.24109995e-01,
         1.94410002e+00,  -1.21140003e-01,  -8.99289995e-02,
        -5.72640002e-01,   2.53939986e-01,   1.99970007e-01,
         5.26599996e-02,   3.35750014e-01,   1.71739995e+00,
        -4.35149997e-01,   2.17390001e-01,   2.21090000e-02,
        -6.64120018e-02,  -3.66420001e-01,  -4.11419988e-01,
        -2.00049996e-01,  -5.14129996e-01,   1.52140006e-01,
         2.87669986e-01,   1.59429997e-01,   5.04710019e-01,
        -4.47310001e-01,   8.42199981e-01,  -4.75209989e-02,
        -2.90859997e-01,  -2.70959996e-02,  -5.63480020e-01,
        -7.71640018e-02,   1.10760003e-01,   2.30910003e-01,
        -5.79890013e-01,   1.91699993e-02,  -2.47799993e-01,
         2.84900010e-01,  -1.42419994e-01,   1.33900002e-01,
         3.67000014e-01,   2.31389999e-01,   1.47330001e-01,
        -2.69219995e-01,

The following cell creates a function that gets the vector of a given string from spaCy

In [81]:
def vec(s): 
    return nlp.vocab[s].vector

In [82]:
vec

<function __main__.vec>

## Cosine similarity and closest neighbors ##
consine() returns cosine similarity of two vector. This is another way to see similarities "more suited to hight-dimensional spaces", like all the words in Frankenstein.

**Have to install numpy**

In [83]:
import numpy as np

In [86]:
from numpy import dot

from numpy.linalg import norm

In [87]:
def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1,v2) / (norm(v1) * norm(v2))
    else:
        return 0.0

In [94]:
cosine(vec('human'), vec('person')) > cosine(vec('dog'), vec ('peace'))

True

In [96]:
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                 key=lambda x: cosine(vec_to_check, vec(x)),
                 reverse=True) [:n]

In [118]:
spacy_closest(tokens, vec("truth"))

['truth',
 'faith',
 'believing',
 'belief',
 'true',
 'believe',
 'Believe',
 'reality',
 'nothing',
 'Nothing']

**Finding the halfway point between two words**

In [119]:
spacy_closest(tokens, meanv([vec("truth"), vec("war")]))

['war',
 'truth',
 'nothing',
 'Nothing',
 'Fear',
 'fear',
 'strife',
 'conflict',
 'wars',
 'humanity']

In [112]:
blue_to_sky = subtractv(vec("men"), vec("god"))
spacy_closest(tokens, addv(blue_to_sky, vec("industry")))

['men',
 'Men',
 'women',
 'business',
 'Among',
 'among',
 'market',
 'professional',
 'opportunities',
 'employees']

blue_to_sky

## Mixing with tracery ##

I'll try to mix what I've done with Tracery rules



In [114]:
import tracery


In [115]:
from tracery.modifiers import base_english

In [116]:
import json

In [120]:
closest_truWar = spacy_closest(tokens, meanv([vec("truth"), vec("war")]))

In [122]:
type (closest_truWar)

list

In [124]:
closest_truWar[6]

'strife'

NameError: name 'random' is not defined

In [127]:
import random

In [135]:
for i in range(20):  
    print ("When will you ever see" + " "+  closest_truWar[i])

When will you ever see war
When will you ever see truth
When will you ever see nothing
When will you ever see Nothing
When will you ever see Fear
When will you ever see fear
When will you ever see strife
When will you ever see conflict
When will you ever see wars
When will you ever see humanity


IndexError: list index out of range

## Adding Dark Souls JSON libraries ##

In [138]:
darksouls_data = json.loads(open("darksouls_edit.json").read())

In [139]:
words = darksouls_data['words']

In [140]:
creatures = words['creatures']
objects = words['objects']
techniques = words['techniques']
actions = words['actions']
techniques = words['techniques']
actions = words['actions']


In [141]:
bodyParts = words['body parts']
attributes = words['attribute']
concepts = words['concepts']
adverbs = json.loads(open("adverbs.json").read())

In [144]:
adverbsList = json.loads(open("adverbs.json").read())

In [145]:
verbsLove = json.loads(open("verbsLove.json").read())

In [146]:
adverbs = adverbsList["adverbs"]

In [148]:
verbs = verbsLove['verb']

In [150]:
conjunctions = [

"when",
"once",
"as soon as",
"if",
"till",
"untill",
"provided that",
"unless",
"even though",
"although",
"as",
"because",
"after",
"before",
"in spite of",
"despite",

]

In [152]:
moods_list = json.loads(open("moods_data.json").read())

In [153]:
moods = moods_list['moods']

In [154]:
locations_list = json.loads(open("locations_data.json").read())

In [155]:
locations = locations_list['locations']

In [170]:
rules = {
    "origin":["#[lover:#creature#][wise:#creature#][muse:#creature#]story#"],
    
    "story": ["One evening in #location#, a #mood# #lover# met #wise.a# who asked: Do you believe that #muse.a# can love #lover.a#?\nThe #wise# looked at the #lover# with #mood.a# #bodyPart# and touched the #lover#'s #bodyPart# while saying:\nOnly when #creature.a# has his or her #bodyPart# #adverb# #verb# by #creature.a#, will #muse.a# truly believe in love."],
   
    "noun": [ "#creature#", "#bodyPart#", "#concept#", "#object#"],
    
"verb": verbs,
"location" : locations,
"object": objects,
"creature": creatures,
"bodyPart": bodyParts,
"action": actions,
"concept": concepts,
"conjunction": conjunctions,
"adverb": adverbs,
"mood" : moods

}
    
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
for i in range(2): 
    print(grammar.flatten("#origin#"))


One evening in Church of Yorshka, a envious queen met a herald who asked: Do you believe that a charmer can love a queen?
The herald looked at the queen with a loving left leg and touched the queen's thumb while saying:
Only when a sellsword has his or her right leg closely kissed by a beanpole, will a charmer truly believe in love.
One evening in Ariandel, a unconcerned beggar met a god who asked: Do you believe that a dwarf can love a beggar?
The god looked at the beggar with an excluded anywhere and touched the beggar's stomach while saying:
Only when an artisan has his or her head painfully rubbed by an artisan, will a dwarf truly believe in love.


## Adding word vectors to Tracery code ##

In [185]:
rules = {
    "origin":["#[lover:#creature#][wise:#creature#][muse:#creature#]story#"],
    
    "story": ["One evening in #location#, a #mood# #lover# met #wise.a# who asked: Do you believe that #muse.a# can love #lover.a#?\nThe #wise# looked at the #lover# with #mood.a# #bodyPart# and touched the #lover#'s #bodyPart# while saying:\nOnly when #creature.a# has his or her #bodyPart# #adverb# #verb# by #creature.a#, will #muse.a# truly believe in love."],
   
    "noun": [ "#creature#", "#bodyPart#", "#concept#", "#object#"],
    
"verb": verbs,
"location" : locations,
"object": objects,
"creature": creatures,
"bodyPart": bodyParts,
"action": actions,
"concept": concepts,
"conjunction": conjunctions,
"adverb": adverbs,
"mood" : moods

}
    
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
for i in range(2): 
    print(grammar.flatten("#origin#"))
    
print(grammar.flatten("#lover#"))
spacy_closest(tokens, vec(grammar.flatten("#lover#")))


One evening in Ariandel, a useless laggard met an ally who asked: Do you believe that a bug can love a laggard?
The ally looked at the laggard with an accomplished arm and touched the laggard's tail while saying:
Only when a Dark Spirit has his or her core carefully caressed by a saint, will a bug truly believe in love.
One evening in Grand Archives, a irked bug met a dragon who asked: Do you believe that a cleric can love a bug?
The dragon looked at the bug with a deprived left arm and touched the bug's right arm while saying:
Only when a dwarf has his or her ringfinger joyously caressed by a Hollow, will a cleric truly believe in love.
bug


['fix',
 'problem',
 'patches',
 'pest',
 'insect',
 'errors',
 'fixed',
 'annoying',
 'worm',
 'code']

In [186]:
spacy_tracery_closest = spacy_closest(tokens, vec(grammar.flatten("#lover#")))

In [188]:
spacy_tracery_closest[0]

'fix'

In [216]:
rules = {
    "origin":["#[lover:#creature#][wise:#creature#][muse:#creature#]story#"],
    
    "story": ["#tracery_halfsies#.......One evening in #location#, a #mood# #lover# met #wise.a# who asked: Do you believe that #muse.a# can love #lover.a#?\nThe #wise# looked at the #lover# with #mood.a# #bodyPart# and touched the #lover#'s #bodyPart# while saying:\nOnly when #creature.a# has his or her #bodyPart# #adverb# #verb# by #creature.a#, will #muse.a# truly believe in love.\n The #lover# looked at a mirror and saw #tracery_lover.a# and #tracery_lover.a# and #tracery_lover.a#...\n Not a single #muse#, only #mood# #tracery_muse# and #tracery_muse.a# and #tracery_muse.a# "],
   
    "noun": [ "#creature#", "#bodyPart#", "#concept#", "#object#"],
    
"verb": verbs,
"location" : locations,
"object": objects,
"creature": creatures,
"bodyPart": bodyParts,
"action": actions,
"concept": concepts,
"conjunction": conjunctions,
"adverb": adverbs,
"mood" : moods,

"tracery_lover": spacy_closest(tokens, vec(grammar.flatten("#lover#"))),
"tracery_muse": spacy_closest(tokens, vec(grammar.flatten("#muse#"))),
"tracery_halfsies": spacy_closest(tokens, meanv([vec("#lover#"), vec("#muse#")]))
}
    
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
for i in range(1): 
    print(grammar.flatten("#origin#"))
    
print(grammar.flatten("#lover#"))
spacy_closest(tokens, vec(grammar.flatten("#lover#")))

spacy_closest(tokens, vec(grammar.flatten("#muse#")))


Immediately.......One evening in Road of Sacrifices , a energetic good fellow met a sniper who asked: Do you believe that a fatty can love a good fellow?
The sniper looked at the good fellow with an outraged core and touched the good fellow's longfinger while saying:
Only when a moneybags has his or her left leg neatly rubbed by an ally, will a fatty truly believe in love.
 The good fellow looked at a mirror and saw an ages and a youth and a Child...
 Not a single fatty, only bummed Agrippa and a deathlike and a Leeuw 
good fellow


['complexions',
 'nourishment',
 'diet',
 'muscle',
 'unwholesome',
 'wholesome',
 'lean',
 'excess',
 'eat',
 'milk']

In [210]:
spacy_closest(tokens, vec(grammar.flatten("lover")))

['lover',
 'lovers',
 'playmate',
 'Friend',
 'friend',
 'husband',
 'kiss',
 'lady',
 'girl',
 'caresses']